# This is a workbook for applying the unitary based on NO

In [2]:
import numpy as np
from dmrghandler.src.dmrghandler.qchem_dmrg_calc import single_qchem_dmrg_calc
from dmrghandler.src.dmrghandler.dmrg_looping import dmrg_central_loop
import dmrghandler.src.dmrghandler.dmrg_calc_prepare as dmrg_calc_prepare
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from block2 import SZ as block2_SZ
from get_dmrg_config import get_dmrg_process_param, get_dmrg_config
from pathlib import Path
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2 import tools
from pyblock2._pyscf.ao2mo import integrals as itg
import pyscf
from eigenstate_check import check_eigenstates
from basic_definitions import chemist_to_physicist
from Hamiltonians.molecular_hamiltonians import get_n2, get_h2o

## 1, Obtain the approximate ground-state

In [19]:
def h1e_hubbard(t, ncas):
    tensor = np.zeros((ncas, ncas))
    for i in range(ncas - 1):
        tensor[i, i + 1] = -t
        tensor[i + 1, i] = -t
    
    return tensor

def g2e_hubbard(U, ncas):
    tensor = np.zeros((ncas, ncas, ncas, ncas))
    for i in range(ncas):
        tensor[i, i, i, i] = U
        
    return tensor


ncas = 4
n_elec = 4
spin = 0

h1e = h1e_hubbard(1, ncas)
g2e = g2e_hubbard(1, ncas)

data = get_n2(bond_length=2.2)

ncas, n_elec, spin, h1e, g2e = data.ncas, data.n_elec, data.spin, data.h1e, data.g2e

### We use the one-body combined Hamiltonian

In [20]:
num_orbitals = ncas
num_electrons = n_elec
num_unpaired = spin
multiplicity = 2 * spin + 1

dmrg_params = get_dmrg_config(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity)

absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(h1e), np.array(g2e), norb=num_orbitals, nelec=num_electrons)
h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), num_orbitals).astype(np.array(h1e).dtype, copy=False)
one_body_dummy = np.zeros((ncas, ncas))
obt, tbt = chemist_to_physicist(one_body_dummy, h_in_tbt)

In [21]:
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-300   , "./mps_directory", verbosity=1, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =       5406

Build MPO | Nsites =    10 | Nterms =       5406 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /    10 .. Mmpo =    26 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.001 T = 0.006
 Site =     1 /    10 .. Mmpo =    66 DW = 0.00e+00 NNZ =      243 SPT = 0.8584 Tmvc = 0.000 T = 0.002
 Site =     2 /    10 .. Mmpo =   110 DW = 0.00e+00 NNZ =      459 SPT = 0.9368 Tmvc = 0.000 T = 0.002
 Site =     3 /    10 .. Mmpo =   138 DW = 0.00e+00 NNZ =     2343 SPT = 0.8457 Tmvc = 0.000 T = 0.005
 Site =     4 /    10 .. Mmpo =   118 DW = 0.00e+00 NNZ =      963 SPT = 0.9409 Tmvc = 0.000 T = 0.002
 Site =     5 /    10 .. Mmpo =   114 DW = 0.00e+00 NNZ =      471 SPT = 0.9650 Tmvc = 0.000 T = 0.001
 Site =     6 /    10 .. Mmpo =   126 DW = 0.00e+00 NNZ =      411 SPT = 0.9714 Tmvc = 0.000 T = 0.002
 Site =     7 /    10 .. Mmpo =    74 DW = 0.00e+00 NNZ =      271 SPT = 0.9709 Tmvc = 0.000 T = 0.001
 Site =     8 /    10 .. Mmpo =

In [22]:
loop_results["finish_reason"]

'Energy change below threshold, limit 1e-300, achieved 0.0'

### Convergence check

In [23]:
symmetry_type = SymmetryTypes.SZ
stack_mem = 10*1024*1024*1024
stack_mem_ratio = 0.5
num_threads = 3
restart_dir=Path("./temp")
mps_dir = Path("./mps_directory/mps_storage/dmrg_loop_030_ket_optimized")
driver = DMRGDriver(
    stack_mem=stack_mem,
    scratch=str(mps_dir),
    clean_scratch=True,  # Default value
    restart_dir=str(restart_dir),
    n_threads=num_threads,
    # n_mkl_threads=n_mkl_threads,  # Default value is 1
    symm_type=symmetry_type,
    mpi=None,  # Default value
    stack_mem_ratio=stack_mem_ratio,  # Default value 0.4
    fp_codec_cutoff=1e-120,  # Default value 1e-16,
)
tools.init(block2_SZ)
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_unpaired, orb_sym=None)
mps = tools.loadMPSfromDir(mps_info=None, mpsSaveDir=mps_dir)
csfs, coeffs = driver.get_csf_coefficients(mps, cutoff=1e-308, iprint=1)
variance = check_eigenstates(num_orbitals, num_electrons, num_unpaired, obt, tbt, "dmrg_loop_030_ket_optimized", "mps_directory")
print(f"Energy variance: {variance}")

cp: mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 and mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 and mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 and mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 and mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.2 and mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_

mps center changed (temporarily)
dtrie finished 0.249319457999718
Number of DET =      14399 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 2222222000  =    0.392773851777078
DET          1 2222220200  =   -0.326027135576614
DET          2 2222202020  =   -0.326027135576000
DET          3 2222200220  =    0.277377350367532
DET          4 22222abba0  =   -0.214670221214525
DET          5 22222baab0  =   -0.214670221213057
DET          6 2222022002  =   -0.198757782609625
DET          7 2222002022  =    0.181174701318047
DET          8 2222020202  =    0.181174701317661
DET          9 2222000222  =   -0.169484458585676
DET         10 2222ab20ba  =   -0.147381757581210
DET         11 2222b2aa0b  =   -0.147381757581144
DET         12 2222ba20ab  =   -0.147381757580549
DET         13 2222a2bb0a  =   -0.147381757580526
DET         14 2222a0bb2a  =    0.129625352414209
DET         15 2222b0aa2b  =    0.129625352414178
DET         16 2222ab02ba  =   

cp: mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 and mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 and mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 and mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 and mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 are identical (not copied).
cp: mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.2 and mps_directory/mps_storage/dmrg_loop_030_ket_optimized/F.MPS.INFO.ket_

Energy variance: 6.607249479556569e-06


## 2, Find the 1RDM and diagonalizes it

In [24]:
pdm = driver.get_1pdm(mps)

In [25]:
# One spin pdm either alpha or beta
rdm = pdm[0]

In [26]:
from scipy.linalg import eigh
import numpy as np
eigenvalues, eigenvectors = eigh(rdm)

# Verify eigenvectors being unitary
product = eigenvectors.conj().T @ eigenvectors
print(np.allclose(product, np.eye(eigenvectors.shape[0])))

True


In [27]:
U_NO = eigenvectors
print(U_NO)

[[-3.68609664e-05 -6.25470444e-13 -4.30674487e-13 -4.73102049e-14
  -2.35546764e-13  6.81224020e-15  1.60870963e-02 -9.78682638e-13
   4.95793384e-11  9.99870594e-01]
 [ 6.18394225e-14 -7.24606639e-13  5.33265672e-13  1.61851270e-12
  -3.56171743e-13  9.06798592e-04  1.09101617e-12  2.46330689e-02
  -9.99696149e-01  4.95772425e-11]
 [ 5.37749877e-12  1.14754156e-13  4.82264232e-14 -5.05939343e-14
   4.02661684e-14  9.09781787e-02 -2.17418548e-11  9.95548669e-01
   2.46133966e-02  1.03366096e-13]
 [ 4.76273902e-02 -2.43410352e-13 -8.92193053e-14 -1.58066975e-14
   1.99775061e-13 -2.75658575e-12  9.98735941e-01  2.17873597e-11
   8.30509101e-13 -1.60670849e-02]
 [ 6.55405654e-11  1.99834753e-12  1.01098404e-12 -6.51692982e-14
   6.14191742e-14  9.95852473e-01  1.60798645e-12 -9.09728543e-02
  -1.33830961e-03 -5.29238272e-14]
 [ 1.12477491e-12 -2.38652860e-10 -5.23846534e-11  9.55017417e-02
  -9.95429263e-01  7.08743509e-14  1.43250993e-13  2.60724279e-14
   5.09899843e-13 -2.32245022e-13

## 3, Combine the one-body and two-body terms

In [28]:
absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(h1e), np.array(g2e), norb=num_orbitals, nelec=num_electrons)
h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), num_orbitals).astype(np.array(h1e).dtype, copy=False)

## 4, Apply the unitary as an orbital rotation

In [29]:
# U_NO = np.array(U_NO).conj().T

In [30]:
p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, h_in_tbt)[0]
orbital_rotated = np.einsum('ak,bl,cm,dn,klmn->abcd', U_NO, U_NO, U_NO, U_NO, h_in_tbt, optimize = p)

In [31]:
print(orbital_rotated.shape)

(10, 10, 10, 10)


In [32]:
num_orbitals = ncas
num_electrons = n_elec
num_unpaired = spin
multiplicity = 2 * spin + 1

dmrg_params = get_dmrg_config(
    num_orbitals, num_electrons, num_unpaired,
    multiplicity)
one_body_dummy = np.zeros((ncas, ncas))
obt, tbt = chemist_to_physicist(one_body_dummy, orbital_rotated)

In [33]:
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-300   , "./mps_directory_2", verbosity=2, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =      17704

Build MPO | Nsites =    10 | Nterms =      17704 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /    10 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.001 T = 0.002
 Site =     1 /    10 .. Mmpo =    82 DW = 0.00e+00 NNZ =      475 SPT = 0.8069 Tmvc = 0.001 T = 0.004
 Site =     2 /    10 .. Mmpo =   126 DW = 0.00e+00 NNZ =      933 SPT = 0.9097 Tmvc = 0.001 T = 0.005
 Site =     3 /    10 .. Mmpo =   186 DW = 0.00e+00 NNZ =     1557 SPT = 0.9336 Tmvc = 0.001 T = 0.005
 Site =     4 /    10 .. Mmpo =   262 DW = 0.00e+00 NNZ =     2165 SPT = 0.9556 Tmvc = 0.001 T = 0.007
 Site =     5 /    10 .. Mmpo =   186 DW = 0.00e+00 NNZ =     8087 SPT = 0.8341 Tmvc = 0.001 T = 0.008
 Site =     6 /    10 .. Mmpo =   126 DW = 0.00e+00 NNZ =     1567 SPT = 0.9331 Tmvc = 0.000 T = 0.002
 Site =     7 /    10 .. Mmpo =    82 DW = 0.00e+00 NNZ =      937 SPT = 0.9093 Tmvc = 0.000 T = 0.002
 Site =     8 /    10 .. Mmpo =

In [72]:
loop_results["finish_reason"]

'Maximum bond dimension reached, limit 1000, achieved 1063'

In [29]:
symmetry_type = SymmetryTypes.SZ
stack_mem = 10*1024*1024*1024
stack_mem_ratio = 0.5
num_threads = 3
restart_dir=Path("./temp")
mps_dir = Path("./mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized")
driver = DMRGDriver(
    stack_mem=stack_mem,
    scratch=str(mps_dir),
    clean_scratch=True,  # Default value
    restart_dir=str(restart_dir),
    n_threads=num_threads,
    # n_mkl_threads=n_mkl_threads,  # Default value is 1
    symm_type=symmetry_type,
    mpi=None,  # Default value
    stack_mem_ratio=stack_mem_ratio,  # Default value 0.4
    fp_codec_cutoff=1e-120,  # Default value 1e-16,
)
tools.init(block2_SZ)
driver.initialize_system(n_sites=num_orbitals, n_elec=num_electrons, spin=num_unpaired, orb_sym=None)
mps = tools.loadMPSfromDir(mps_info=None, mpsSaveDir=mps_dir)
csfs, coeffs = driver.get_csf_coefficients(mps, cutoff=1e-308, iprint=1)
variance = check_eigenstates(num_orbitals, num_electrons, num_unpaired, obt, tbt, "dmrg_loop_011_ket_optimized", "mps_directory_2")
print(f"Energy variance: {variance}")

cp: mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 and mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 are identical (not copied).
cp: mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 and mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 are identical (not copied).
cp: mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 and mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 are identical (not copied).
cp: mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 and mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 are identical (not copied).
cp: mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.2 and mps_directory_2/mps_storage/dmrg_loop_011_ket_optim

mps center changed (temporarily)
dtrie finished 0.0030290000004242756
Number of DET =         36 (cutoff =    1e-308)
Sum of weights of included DET =    1.000000000000000

DET          0 0022  =    0.985989514774742
DET          1 0202  =   -0.117719847122073
DET          2 abba  =   -0.049072271627202
DET          3 baab  =   -0.049072271627202
DET          4 2020  =   -0.046107192302663
DET          5 baba  =    0.043859516513709
DET          6 abab  =    0.043859516513709
DET          7 0220  =   -0.038646761398873
DET          8 2002  =   -0.038646761398726
DET          9 2200  =    0.011274917209435
DET         10 bbaa  =    0.005212755113513
DET         11 aabb  =    0.005212755113513
DET         12 2b0a  =    0.001477419730685
DET         13 2a0b  =   -0.001477419730685
DET         14 b2a0  =   -0.001477419730671
DET         15 a2b0  =    0.001477419730671
DET         16 b0a2  =    0.000039697448018
DET         17 a0b2  =   -0.000039697448018
DET         18 0a2b  =    0.0000396

cp: mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 and mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.0 are identical (not copied).
cp: mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 and mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.0 are identical (not copied).
cp: mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 and mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.1 are identical (not copied).
cp: mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 and mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.LEFT.1 are identical (not copied).
cp: mps_directory_2/mps_storage/dmrg_loop_011_ket_optimized/F.MPS.INFO.ket_optimized.RIGHT.2 and mps_directory_2/mps_storage/dmrg_loop_011_ket_optim

Energy variance: 0.0
